In [1]:
#Importing modules:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as dsets
from torch.autograd import Variable

In [2]:
#Creating FashionMNIST Dataset Class:
from __future__ import print_function
import torch.utils.data as data
from PIL import Image
import os
import os.path
import errno
import torch
import codecs


class FashionMNIST(data.Dataset):
    """`MNIST <http://yann.lecun.com/exdb/mnist/>`_ Dataset.
    Args:
        root (string): Root directory of dataset where ``processed/training.pt``
            and  ``processed/test.pt`` exist.
        train (bool, optional): If True, creates dataset from ``training.pt``,
            otherwise from ``test.pt``.
        download (bool, optional): If true, downloads the dataset from the internet and
            puts it in root directory. If dataset is already downloaded, it is not
            downloaded again.
        transform (callable, optional): A function/transform that  takes in an PIL image
            and returns a transformed version. E.g, ``transforms.RandomCrop``
        target_transform (callable, optional): A function/transform that takes in the
            target and transforms it.
    """
    urls = [
        'http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/train-images-idx3-ubyte.gz',
        'http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/train-labels-idx1-ubyte.gz',
        'http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/t10k-images-idx3-ubyte.gz',
        'http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/t10k-labels-idx1-ubyte.gz',
    ]
    raw_folder = 'raw'
    processed_folder = 'processed'
    training_file = 'training.pt'
    test_file = 'test.pt'

    def __init__(self, root, train=True, transform=None, target_transform=None, download=False):
        self.root = os.path.expanduser(root)
        self.transform = transform
        self.target_transform = target_transform
        self.train = train  # training set or test set

        if download:
            self.download()

        if not self._check_exists():
            raise RuntimeError('Dataset not found.' +
                               ' You can use download=True to download it')

        if self.train:
            self.train_data, self.train_labels = torch.load(
                os.path.join(root, self.processed_folder, self.training_file))
        else:
            self.test_data, self.test_labels = torch.load(os.path.join(root, self.processed_folder, self.test_file))

    def __getitem__(self, index):
        """
        Args:
            index (int): Index
        Returns:
            tuple: (image, target) where target is index of the target class.
        """
        if self.train:
            img, target = self.train_data[index], self.train_labels[index]
        else:
            img, target = self.test_data[index], self.test_labels[index]

        # doing this so that it is consistent with all other datasets
        # to return a PIL Image
        img = Image.fromarray(img.numpy(), mode='L')

        if self.transform is not None:
            img = self.transform(img)

        if self.target_transform is not None:
            target = self.target_transform(target)

        return img, target

    def __len__(self):
        if self.train:
            return len(self.train_data)
        else:
            return len(self.test_data)

    def _check_exists(self):
        return os.path.exists(os.path.join(self.root, self.processed_folder, self.training_file)) and \
            os.path.exists(os.path.join(self.root, self.processed_folder, self.test_file))

    def download(self):
        """Download the MNIST data if it doesn't exist in processed_folder already."""
        from six.moves import urllib
        import gzip

        if self._check_exists():
            return

        # download files
        try:
            os.makedirs(os.path.join(self.root, self.raw_folder))
            os.makedirs(os.path.join(self.root, self.processed_folder))
        except OSError as e:
            if e.errno == errno.EEXIST:
                pass
            else:
                raise

        for url in self.urls:
            print('Downloading ' + url)
            data = urllib.request.urlopen(url)
            filename = url.rpartition('/')[2]
            file_path = os.path.join(self.root, self.raw_folder, filename)
            with open(file_path, 'wb') as f:
                f.write(data.read())
            with open(file_path.replace('.gz', ''), 'wb') as out_f, \
                    gzip.GzipFile(file_path) as zip_f:
                out_f.write(zip_f.read())
            os.unlink(file_path)

        # process and save as torch files
        print('Processing...')

        training_set = (
            read_image_file(os.path.join(self.root, self.raw_folder, 'train-images-idx3-ubyte')),
            read_label_file(os.path.join(self.root, self.raw_folder, 'train-labels-idx1-ubyte'))
        )
        test_set = (
            read_image_file(os.path.join(self.root, self.raw_folder, 't10k-images-idx3-ubyte')),
            read_label_file(os.path.join(self.root, self.raw_folder, 't10k-labels-idx1-ubyte'))
        )
        with open(os.path.join(self.root, self.processed_folder, self.training_file), 'wb') as f:
            torch.save(training_set, f)
        with open(os.path.join(self.root, self.processed_folder, self.test_file), 'wb') as f:
            torch.save(test_set, f)

        print('Done!')


def get_int(b):
    return int(codecs.encode(b, 'hex'), 16)


def parse_byte(b):
    if isinstance(b, str):
        return ord(b)
    return b


def read_label_file(path):
    with open(path, 'rb') as f:
        data = f.read()
        assert get_int(data[:4]) == 2049
        length = get_int(data[4:8])
        labels = [parse_byte(b) for b in data[8:]]
        assert len(labels) == length
        return torch.LongTensor(labels)


def read_image_file(path):
    with open(path, 'rb') as f:
        data = f.read()
        assert get_int(data[:4]) == 2051
        length = get_int(data[4:8])
        num_rows = get_int(data[8:12])
        num_cols = get_int(data[12:16])
        images = []
        idx = 16
        for l in range(length):
            img = []
            images.append(img)
            for r in range(num_rows):
                row = []
                img.append(row)
                for c in range(num_cols):
                    row.append(parse_byte(data[idx]))
                    idx += 1
        assert len(images) == length
    return torch.ByteTensor(images).view(-1, 28, 28)

**Creating training and test datasets:**

In [3]:
train_dataset = FashionMNIST(root='./data',
                           train = True,
                           transform = transforms.ToTensor(),        
                           download = True)

In [4]:
test_dataset = FashionMNIST(root = './data',
                                 train = False,
                                 transform = transforms.ToTensor())

**Getting sizes of training and test datasets' images and labels:**

In [5]:
print(train_dataset.train_data.size())

torch.Size([60000, 28, 28])


In [6]:
print(train_dataset.train_labels.size())

torch.Size([60000])


In [7]:
print(test_dataset.test_data.size())

torch.Size([10000, 28, 28])


In [8]:
print(test_dataset.test_labels.size())

torch.Size([10000])


**Making Dataset Iterable:**

In [9]:
batch_size = 60
n_iters = 24000
num_epochs = int(n_iters / (len(train_dataset) / batch_size))

train_loader = torch.utils.data.DataLoader(dataset = train_dataset,
                                          batch_size = batch_size,
                                          shuffle = True)

test_loader = torch.utils.data.DataLoader(dataset = test_dataset,
                                          batch_size = batch_size,
                                          shuffle = False)


**Creating CNN Model Class:**

In [10]:
class CNNModule(nn.Module):
    def __init__(self):
        super (CNNModule,self).__init__()
        
        #Convolution 1
        self.cnn1 = nn.Conv2d(in_channels=1,out_channels=16,kernel_size=5,stride=1,padding=2)
        self.relu1=nn.ReLU()
        
        #We apply Xavier initialization as it helps in:
        #Keeping the signal in a reasonable range of values through many layers.
        #Going deep into the network 
        #So that the inputs of each activation function fall within the range of the activation function.
        nn.init.xavier_uniform_(self.cnn1.weight)
        
        #Maxpool 1    
        self.maxpool1=nn.MaxPool2d(kernel_size=2)
        
        #Convolution 2
        self.cnn2=nn.Conv2d(in_channels=16,out_channels=32,kernel_size=5,stride=1,padding=2)
        self.relu2=nn.ReLU()
        nn.init.xavier_uniform_(self.cnn2.weight)

        #Maxpool 2
        self.maxpool2=nn.MaxPool2d(kernel_size=2)
        
        #Fully Connected 1(readout layer)
        self.fcl=nn.Linear(32*7*7,10)
    def forward(self,x):
        
        #Convolution 1
        out=self.cnn1(x)
        out=self.relu1(out)
        #print ("CNN1")
        #print (out.size())
        
        #Maxpool 1  
        out=self.maxpool1(out)
        #print ("Maxpool1")
        #print (out.size())
        
        #Convolution 2
        out=self.cnn2(out)
        out=self.relu2(out)
        #print ("CNN2")
        #print (out.size())
        
        #Maxpool 2
        out=self.maxpool2(out)
        #print ("Maxpool2")
        #print (out.size(0))
        
        #Resize
        #Original out size:(100,32,7,7)
        #out.size(0):100
        #New out size:(100,32*7*7)
        out=out.view(out.size(0),-1)
        
        #Linear function(readout)
        out=self.fcl(out)

        return out

**Instantiating Model Class:**

In [11]:
model = CNNModule()

**Instantiating Optimizer Class:**

In [12]:
criterion = nn.CrossEntropyLoss()

In [13]:
#Setting adequate learning rate:
learning_rate = 0.1

optimizer = torch.optim.SGD(model.parameters(),lr = learning_rate)

**Parameters in Depth:**

In [14]:
print(model.parameters())

print(len(list(model.parameters())))

#Convolution 1:16 Kernels
print(list(model.parameters())[0].size())

#Convolution 1 bias:16 Kernels
print(list(model.parameters())[1].size())

#Convolution 2 :32 Kernels with depth 16
print(list(model.parameters())[2].size())

#Convolution 2 bias:32 Kernels with depth 16
print(list(model.parameters())[3].size())

#Fully Connected Layer 1
print(list(model.parameters())[4].size())

#Fully Connected Layer 1 bias
print(list(model.parameters())[5].size())

<generator object Module.parameters at 0x7fedc2edce60>
6
torch.Size([16, 1, 5, 5])
torch.Size([16])
torch.Size([32, 16, 5, 5])
torch.Size([32])
torch.Size([10, 1568])
torch.Size([10])


**Training The Model:**

In [ ]:
iter = 0
for epoch in range(num_epochs):                 #num_epochs = 5
    for i,(images,labels) in enumerate (train_loader): #for all 60000 images
        
        #Loading images as variables
        images=Variable(images)
        labels=Variable(labels)
        
        #Clearing gradients wrt parameters
        optimizer.zero_grad()
        
        #Forward pass to get outputs/logits
        outputs=model(images)
        
        #Calculate loss-->softmax to cross entropy loss
        loss=criterion(outputs,labels)
        
        #Getting gradients wrt parameters
        loss.backward()
        
        #Updating parameters
        optimizer.step()
        
        
        iter+=1
        if iter%500==0:
            correct=0
            total=0
            #Iterating through test dataset:
            for images,labels in test_loader:
                images=Variable(images)

                outputs=model(images)
                
                _,predicted=torch.max(outputs.data,1)
                total+=labels.size(0)
                correct+=(predicted==labels).sum()
                
            #Getting accuracy:    
            accuracy= (100.0* correct)/(total)
            print("Iterations: {},  Loss: {} , Accuracy:{}".format(iter,loss.data[0],accuracy))

/home/kunal/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:41: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


Iterations: 500,  Loss: 0.42539894580841064 , Accuracy:83
Iterations: 1000,  Loss: 0.4683968424797058 , Accuracy:85
Iterations: 1500,  Loss: 0.3467230498790741 , Accuracy:85
Iterations: 2000,  Loss: 0.46919000148773193 , Accuracy:87
Iterations: 2500,  Loss: 0.22110550105571747 , Accuracy:88
Iterations: 3000,  Loss: 0.30021020770072937 , Accuracy:88
Iterations: 3500,  Loss: 0.26187974214553833 , Accuracy:88
Iterations: 4000,  Loss: 0.2867961525917053 , Accuracy:89
Iterations: 4500,  Loss: 0.31819021701812744 , Accuracy:89
Iterations: 5000,  Loss: 0.251635879278183 , Accuracy:88
